### Scanning 3D Materials and Creating Thresholds

#### What is a 3D Scan?
A 3D scan is a digital representation of the three-dimensional shape of an object. This technology captures the exact size and shape of physical objects using a device that records comprehensive data on its shape, color, and sometimes texture. The resulting scan forms a point cloud, or a collection of data points in three-dimensional space, which can be used to create a 3D model. These scans are utilized across various industries, including manufacturing, entertainment, healthcare, and archaeology, to create models for analysis, reproduction, or digital rendering.

<center>
    <img src="img/01.jpg" alt="Alt Text" width="400"/>
</center>

#### Objective
The described training module is focused on several key skills necessary for working effectively with 3D scans:

#### - Cleaning Noise from 3D Scans:
In this context, noise in a 3D scan refers to any data that does not pertain to the geometry of interest, similar to cropping out unwanted areas in a photograph. This might include removing extraneous elements like the base plate on which a 3D printed part sits, or any supports used to keep the part upright during scanning. The goal is to isolate and enhance the scan data of the specific object of interest, ensuring cleaner and more accurate geometric analysis.

#### - Calibrating the Scan:
We utilize mathematical techniques to refine the scanner’s accuracy. This involves applying linear regression to a set of calibration data points to determine the best-fit line. From this, you calculate the slope, which helps in adjusting and setting the scanner’s parameters effectively.

#### - Fitting Geometric Shapes and Obtaining Useful Parameters:
In many applications, especially in industrial and engineering settings, fitting geometric shapes to 3D data helps in analyzing the object’s characteristics or in designing parts that need to interact with the scanned object. This part of the module teaches how to apply mathematical models to fit shapes such as rectangles, ellipses, and circles to the scan data. 

#### Press ▶️ to Select 3D Data to Load

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.stats import linregress
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import Dropdown, interact
from IPython.display import clear_output
from functions import*  # Assuming this imports necessary custom functions

# Function to get a list of CSV files in the 'data' folder
def get_csv_files():
    data_dir = './data'  # Directory where CSV files are located
    return [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]

# Function to load the selected CSV file into a global variable
def load_selected_csv(file):
    global data
    clear_output(wait=True)
    print(f"Loading {file}...")
    data = load_data(file)  # Assuming load_data is a custom function to load CSV into data
    plot(data)  # Assuming plot is a custom function to plot data

# Dropdown widget to select a CSV file
csv_files = get_csv_files()
file_selector = Dropdown(options=csv_files, description="Select file:")

# Interactive widget to update the global variable based on selected file
interact(load_selected_csv, file=file_selector);


### Plot Calibration Instructions

- Select Bed Region: Use the sliders to accurately define the region that represents the bed in the scan or plot. This area should be flat and stable.



- Fit Linear Regression: Once the bed region is selected, a linear regression line is fitted to this data. This line helps determine the slope of the bed.

- Apply Slope Calibration: The slope obtained from the linear regression of the bed region is then applied to the entire scan or plot. This calibration ensures that any inclinations or biases in the entire data set are corrected based on the flat bed data.


<center>
    <img src="img/02.jpg" alt="Alt Text" width="800"/>
</center>

#### Press ▶️ to Select Bed Region for Calibration <font color = 'green'>(Scan Bed)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import FloatSlider, interactive
from IPython.display import display

# Assuming 'data' is your original 2D numpy array
data_temp = np.copy(data)  # Use the original data directly with NaNs preserved

# Flatten the data to prepare for the scatter plot
points = data_temp.flatten()

# Initialize global variables
global upper_bound_plate, lower_bound_plate
upper_bound_plate = np.nanmax(points)
lower_bound_plate = np.nanmin(points)

def update_plots(y1, y2):
    global upper_bound_plate, lower_bound_plate
    # Update global variables with the slider values
    upper_bound_plate = y2
    lower_bound_plate = y1
    
    # Create a mask for values outside the slider range, respecting NaNs
    mask = (data_temp < y1) | (data_temp > y2)
    filtered_data = np.copy(data_temp)
    filtered_data[mask] = np.nan  # Set values outside the range to NaN
    
    # Plotting using subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    # Scatter plot on the first subplot
    valid_points = ~np.isnan(points)  # Mask to remove NaNs for the scatter plot
    ax1.scatter(np.arange(len(points))[valid_points], points[valid_points], marker='o', linestyle='-', s=0.01)
    ax1.set_title('Flattened Array Values')
    ax1.set_xlabel('Index')
    ax1.set_ylabel('Height Value')
    ax1.grid(True)
    ax1.axhline(y=lower_bound_plate, color='r', linestyle='--')
    ax1.axhline(y=upper_bound_plate, color='g', linestyle='--')
    
    # Heatmap on the second subplot
    sns.heatmap(filtered_data, cmap='viridis', cbar=False, ax=ax2)
    ax2.set_aspect(aspect= 'equal')
    ax2.set_title('Filtered Data Heatmap')
    ax2.set_xticks([])
    ax2.set_yticks([])
    
    plt.tight_layout()
    plt.show()

# Set up sliders for the interactive lines
y1_slider = FloatSlider(min=np.nanmin(points), max=np.nanmax(points), step=0.01, value=np.nanmin(points), description='Minimum')
y2_slider = FloatSlider(min=np.nanmin(points), max=np.nanmax(points), step=0.01, value=np.nanmax(points), description='Maximum')

interactive_plot = interactive(update_plots, y1=y1_slider, y2=y2_slider)
output = interactive_plot.children[-1]
display(interactive_plot)


#### Press ▶️ to Start Calibration

In [ ]:
import ipywidgets as widgets
from IPython.display import display

button = widgets.Button(description="Calibrate Scan")
output = widgets.Output()

def on_button_clicked(b):
    global data  # Make sure to modify the global 'data' variable
    with output:
        data = correct_tilt(data, lower_bound_plate, upper_bound_plate)
        print("Data has been updated.")

button.on_click(on_button_clicked)
display(button, output)

### Feature Extraction Instructions
- **Define Area**: Carefully adjust the sliders to encompass the area of the plot that contains the part of interest.

#### Press ▶️ to select region of interest for feature extraction: <font color = 'red'>(Part)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import FloatSlider, interactive
from IPython.display import display

# Assuming 'data' is your original 2D numpy array
data_temp = np.copy(data)  # Use the original data directly with NaNs preserved

# Flatten the data to prepare for the scatter plot
points = data_temp.flatten()

# Initialize global variables
global upper_bound, lower_bound
upper_bound = np.nanmax(points)
lower_bound = np.nanmin(points)

def update_plots(y1, y2):
    global upper_bound, lower_bound
    # Update global variables with the slider values
    upper_bound = y2
    lower_bound = y1
    
    # Create a mask for values outside the slider range, respecting NaNs
    mask = (data_temp < y1) | (data_temp > y2)
    filtered_data = np.copy(data_temp)
    filtered_data[mask] = np.nan  # Set values outside the range to NaN
    
    # Plotting using subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    # Scatter plot on the first subplot
    valid_points = ~np.isnan(points)  # Mask to remove NaNs for the scatter plot
    ax1.scatter(np.arange(len(points))[valid_points], points[valid_points], marker='o', linestyle='-', s=0.01)
    ax1.set_title('Flattened Array Values')
    ax1.set_xlabel('Index')
    ax1.set_ylabel('Height Value')
    ax1.grid(True)
    ax1.axhline(y=lower_bound, color='r', linestyle='--')
    ax1.axhline(y=upper_bound, color='g', linestyle='--')
    
    # Heatmap on the second subplot
    sns.heatmap(filtered_data, cmap='viridis', cbar=False, ax=ax2)
    ax2.set_aspect(aspect= 'equal')
    ax2.set_title('Filtered Data Heatmap')
    ax2.set_xticks([])
    ax2.set_yticks([])
    
    plt.tight_layout()
    plt.show()

# Set up sliders for the interactive lines
y1_slider = FloatSlider(min=np.nanmin(points), max=np.nanmax(points), step=0.01, value=np.nanmin(points), description='Minimum')
y2_slider = FloatSlider(min=np.nanmin(points), max=np.nanmax(points), step=0.01, value=np.nanmax(points), description='Maximum')

interactive_plot = interactive(update_plots, y1=y1_slider, y2=y2_slider)
output = interactive_plot.children[-1]
display(interactive_plot)


### Rationale for Image Conversion in Feature Extraction

- **Conversion to Image**: The 3D scan data is transformed into an image format. This step is crucial for facilitating subsequent analysis.
- **Utilizing Image Processing Tools**: The decision to convert the scan to an image is driven by the availability of advanced, efficient tools within the image processing community. These tools are well-developed and widely used for extracting and fitting shapes.


#### Press ▶️ to select feature

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Ellipse
import ipywidgets as widgets
from IPython.display import display

# Define the scale factor (0.02 mm per pixel)
scale_factor = 0.02

# Define the function to execute upon shape and epsilon selection
def fit_and_plot(shape, epsilon_factor):
    binary_data = np.where((data >= lower_bound) & (data <= upper_bound), 1, 0)
    contours, _ = cv2.findContours(binary_data.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        epsilon = epsilon_factor * cv2.arcLength(largest_contour, True)
        approx = cv2.approxPolyDP(largest_contour, epsilon, True)

        plt.figure(figsize=(6, 6))
        plt.imshow(binary_data, cmap='gray', origin='lower', extent=[0, binary_data.shape[1] * scale_factor, 0, binary_data.shape[0] * scale_factor])
        
        if shape == 'Rectangle':
            rect = cv2.minAreaRect(approx)
            box = cv2.boxPoints(rect) * scale_factor
            plt.gca().add_patch(Polygon(box, closed=True, color='red', fill=False, linewidth=2))
            print(f"Rectangle width: {rect[1][0] * scale_factor} mm, height: {rect[1][1] * scale_factor} mm")
        
        elif shape == 'Triangle':
            triangle = cv2.minEnclosingTriangle(approx)[1] * scale_factor
            plt.gca().add_patch(Polygon(triangle[0], closed=True, color='blue', fill=False, linewidth=2))
            print("Triangle vertices:", triangle[0])
        
        elif shape == 'Square':
            rect = cv2.minAreaRect(approx)
            side = max(rect[1]) * scale_factor
            center = np.array(rect[0]) * scale_factor
            box = np.array([
                [center[0] - side / 2, center[1] - side / 2],
                [center[0] + side / 2, center[1] - side / 2],
                [center[0] + side / 2, center[1] + side / 2],
                [center[0] - side / 2, center[1] + side / 2],
            ])
            plt.gca().add_patch(Polygon(box, closed=True, color='green', fill=False, linewidth=2))
            print(f"Square side length: {side} mm")
        
        elif shape == 'Circle':
            center, radius = cv2.minEnclosingCircle(approx)
            center_scaled = np.array(center) * scale_factor
            radius_scaled = radius * scale_factor
            circle_patch = plt.Circle(center_scaled, radius_scaled, color='orange', fill=False, linewidth=2)
            plt.gca().add_patch(circle_patch)
            print(f"Circle radius: {radius_scaled} mm")
        
        elif shape == 'Ellipse':
            if len(approx) >= 5:  # Ensure there are at least 5 points to fit an ellipse
                ellipse = cv2.fitEllipse(approx)
                ellipse_center = np.array(ellipse[0]) * scale_factor
                axes = np.array(ellipse[1]) * scale_factor
                angle = ellipse[2]
                plt.gca().add_patch(Ellipse(xy=ellipse_center, width=axes[0], height=axes[1], angle=angle, edgecolor='purple', fill=False, linewidth=2))
                print(f"Ellipse center: {ellipse_center} mm, axes: {axes} mm, angle: {angle}")
            else:
                print("Not enough points to fit an ellipse.")


        plt.xlabel('X (mm)')
        plt.ylabel('Y (mm)')
        plt.show()
    else:
        print("No contours found in the binary image.")

# Widgets for shape and epsilon factor selection
shape_selector = widgets.Dropdown(options=['Rectangle', 'Triangle', 'Square', 'Ellipse', 'Circle'], description='Shape:')
epsilon_slider = widgets.FloatSlider(value=0.01, min=0.0, max=0.1, step=0.005, description='Epsilon Factor:', readout_format='.3f')

# Link widgets to the function
interactive_plot = widgets.interactive_output(fit_and_plot, {'shape': shape_selector, 'epsilon_factor': epsilon_slider})

# Display the widgets and the plot
display(widgets.VBox([shape_selector, epsilon_slider]), interactive_plot)


## [🏠 Home](../../../welcomePage.ipynb)